<a href="https://colab.research.google.com/github/chervonij/DFL-Colab/blob/master/DFL_Colab_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to DFL-Colab!

It's a limited  demo of original DeepFaceLab.

# Overview


*   Extractor works in full functionality.
*   Training work without preview (for now).
*   Training log output fixed.
*   Converter works in full functionality.
*   You can import/export workspace with your Google Drive.
*   Google Colab machine active for 12 hours, so don't forget to export workspace



# Step 1. Clone Github repository and install requirements.


*   After requirements wil be install you need restart runtime (you will see button in output)
*   Pull updates is an optional function if you need get update from repository



In [0]:
#Clone stable master branch(recommended)

!git clone https://github.com/chervonij/DFL-Colab.git

In [0]:
#Pull updates from repository

%cd /content/DFL-Colab
!git pull

In [0]:
#Install requirements

!pip install -r /content/DFL-Colab/requirements.txt
!pip install --upgrade scikit-image

# Step 2. Upload workspace from Google Drive

*   Workspace must be named "workspace.zip"
*   Inside archive you have a dir "workspace"



In [0]:
#Mount Google Drive as folder

from google.colab import drive

drive.mount('/content/drive', force_remount=True)

In [0]:
#Copy zip archive of your workspace from Drive

!cp /content/drive/My\ Drive/workspace.zip /content/DFL-Colab
%cd "DFL-Colab"

!unzip workspace.zip

# Step 3. Extract faces

*   Detect and align faces with S3FD detector
*   Export workspace to Google Drive after extract and sort it manually 


In [0]:
!cd /content/DFL-Colab/

In [0]:
#Extract frames from SRC video

!python main.py videoed extract-video --input-file workspace/data_src.mp4 --output-dir workspace/data_src/

In [0]:
#Extract frames from DST video

!python main.py videoed extract-video --input-file workspace/data_dst.mp4 --output-dir workspace/data_dst/

In [0]:
#Extract data_src

!python main.py extract --input-dir workspace/data_src --output-dir workspace/data_src/aligned --detector s3fd --debug

In [0]:
#Extract data_dst

!python main.py extract --input-dir workspace/data_dst --output-dir workspace/data_dst/aligned --detector s3fd --debug

In [0]:
#Sort by histogram SRC faces

!python main.py sort --input-dir workspace/data_src/aligned --by hist

In [0]:
#Sort by histogram DST faces

!python main.py sort --input-dir workspace/data_dst/aligned --by hist

In [0]:
#Zip and copy workspace.zip to Google Drive

!zip -r workspace.zip workspace
!cp  /content/DFL-Colab/workspace_zip.zip /content/drive/My\ Drive/

# Step 4. Train model

*   Only SAE
*   Set model options on output field
*   You can't get preview in output. (for now)
*   Different samples are saved in SAE_history. You can see the last saved preview, if you run the block 'Show last preview ..', ut you have to stop training for this.
*   Also, you can see the preview if you download the image from SAE_history manually. It does not require to stop training.
*   I plan to make a preview for the notebook later
*   For stop training you  need manually interrupt execution. You get error in output, but model will be saved.
*   **DON'T FORGET TO EXPORT WORKSPACE**


In [0]:
!python main.py train --training-data-src-dir workspace/data_src/aligned --training-data-dst-dir workspace/data_dst/aligned --model-dir workspace/model --model SAE --no-preview

In [0]:
#@title #Show last preview from history { vertical-output: true }

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as img
import glob
import os
from IPython.display import clear_output

list_of_files = glob.glob('/content/DFL-Colab/workspace/model/SSAE_history/*') # * means all if need specific format then *.csv
img_path = max(list_of_files, key=os.path.getctime)

clear_output()
imdata = img.imread(img_path)
plt.figure(figsize=(10,10))
plt.axis('off')
plt.imshow(imdata)
plt.show()


# Step 5. Convert frames

In [0]:
!python main.py convert --input-dir workspace/data_dst --output-dir workspace/data_dst/merged --aligned-dir workspace/data_dst/aligned --model-dir workspace/model --model SAE

In [0]:
#If you need, you can zip your converted frames and export to Drive

!zip -r workspace_merged.zip workspace/data_dst/merged
!cp /content/DFL-Colab/workspace_merged.zip /content/drive/My\ Drive/

In [0]:
#Convert frames to result video 

!python main.py videoed video-from-sequence --input-dir workspace/data_dst/merged --output-file workspace/result.mp4 --reference-file workspace/data_dst.mp4

In [0]:
#Copy result video to Drive
!cp /content/DFL-Colab/workspace/result.mp4 /content/drive/My\ Drive/

# Zip workspace and export to Drive

*   In this block you can export your workspace to Drive
*   Don't forget backup your workspace, because Colab time limit is 12 hours
*   Also, you can delete your model if something goes wrong



In [0]:
#Zip workspace and copy to your Drive

!zip -r workspace.zip workspace
!cp  /content/DFL-Colab/workspace.zip /content/drive/My\ Drive/

In [0]:
#Delete your model if you need it

!rm -R workspace/model/*

In [0]:
#Delete SAE history images

!rm workspace/model/SAE_history/*